In [35]:
SEASON = '2003-04'

In [36]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Kevin Garnett,MIN,PF,140.010782,27,82,82,39.4,9.8,19.6,...,29.6,10.4,8.0,18.3,0.272,6.8,3.3,10.2,10.0,58.0
2,Tim Duncan,SAS,PF,99.582424,27,69,68,36.6,8.6,17.1,...,29.7,5.9,7.2,13.1,0.249,5.2,3.3,8.5,6.7,57.0
3,Andrei Kirilenko,UTA,PF,77.199273,22,78,78,37.1,5.3,11.9,...,21.9,6.5,5.0,11.6,0.192,5.0,2.9,7.9,7.2,42.0
4,Kobe Bryant,LAL,SG,72.601963,25,65,64,37.6,7.9,18.1,...,29.1,7.8,3.0,10.7,0.210,5.1,0.5,5.6,4.7,56.0
5,Peja Stojaković,SAC,SF,71.582478,26,81,81,40.3,8.2,17.1,...,23.9,11.4,2.1,13.5,0.198,5.2,-1.3,3.9,4.9,55.0
6,Shaquille O'Neal,LAL,C,69.043894,31,67,67,36.8,8.3,14.1,...,26.3,5.9,4.0,9.9,0.192,3.8,1.0,4.8,4.1,56.0
7,Sam Cassell,MIN,PG,68.187376,34,81,81,35.0,7.7,15.7,...,27.2,8.8,3.3,12.1,0.205,4.1,0.3,4.4,4.6,58.0
8,Dirk Nowitzki,DAL,C,65.086822,25,77,77,37.9,7.9,17.0,...,24.5,9.0,2.5,11.5,0.190,3.8,0.0,3.8,4.3,52.0
9,Tracy McGrady,ORL,SG,58.962797,24,67,67,39.9,9.7,23.4,...,33.2,8.1,0.4,8.4,0.151,7.6,-1.2,6.4,5.7,21.0
10,Baron Davis,NOH,PG,58.139776,24,67,66,40.1,8.3,20.9,...,30.1,4.3,3.2,7.5,0.133,4.7,0.5,5.2,4.8,41.0
